In [2]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from typing import *

In [3]:
from pandas.io.json import json_normalize

In [4]:
idealista_data_url = 'https://www.idealista.com/data/'

In [5]:
response = requests.get(idealista_data_url)

In [6]:
RE_JS_DATA = re.compile(
    r'var\s+home_js_model\s*=\s*(?P<data>{.*?});\s*</script>',
    re.DOTALL | re.MULTILINE,
)

In [7]:
data_string = re.search(RE_JS_DATA, response.text).group('data')

In [8]:
data = json.loads(data_string)

In [9]:
data.keys()

dict_keys(['stock', 'summary', 'prices', 'evolution', 'ranking', 'evolutionXZones', 'province', 'countryInitials', 'nameProvince', 'nameCountry'])

In [10]:
def write_json(data: dict, filename: str):
    with open(filename, 'w') as fp:
        json.dump(data, fp, indent=4, ensure_ascii=False)

In [11]:
write_json(data, 'idealista_data_raw.json')

In [24]:
df_nested = pd.DataFrame.from_dict(data['prices'], orient='index').reset_index()
df_nested.head()

,index,wkt,province,latitude,longitude,price,priceThreeMTranslations,diferenceTranslations,color,closeDate,chart,province_chart,country_chart
0,0-EU-ES-15,"{""type"":""MultiPolygon"",""coordinates"":[[[[-9.30...",A Coruña,43.152465,-8.481585,"1,247","{'es': '-0,20%', 'en': '-0.20%', 'it': '-0,20%...","{'es': '-2', 'en': '-2', 'it': '-2', 'pt': '-2'}",#b1d004,31 de Julio,"[{'title': 'Días en mercado', 'value': 177}, {...",NaN,NaN
1,0-EU-IT-AG,"{""type"":""MultiPolygon"",""coordinates"":[[[[12.31...",Agrigento,36.617789999999,13.179265,846,"{'es': '-3,74%', 'en': '-3.74%', 'it': '-3,74%...","{'es': '-32', 'en': '-32', 'it': '-32', 'pt': ...",#cbe631,31 de Julio,"[{'title': 'Días en mercado', 'value': 45636},...",NaN,NaN
2,0-EU-ES-02,"{""type"":""MultiPolygon"",""coordinates"":[[[[-2.88...",Albacete,38.722695,-1.899265,"1,042","{'es': '-0,50%', 'en': '-0.50%', 'it': '-0,50%...","{'es': '-5', 'en': '-5', 'it': '-5', 'pt': '-5'}",#cbe631,31 de Julio,"[{'title': 'Días en mercado', 'value': 188}, {...",NaN,NaN
3,0-EU-IT-AL,"{""type"":""MultiPolygon"",""coordinates"":[[[[8.109...",Alessandria,44.835174999999,8.6618049999998,839,"{'es': '-1,57%', 'en': '-1.57%', 'it': '-1,57%...","{'es': '-13', 'en': '-13', 'it': '-13', 'pt': ...",#cbe631,31 de Julio,"[{'title': 'Días en mercado', 'value': 45632},...",NaN,NaN
4,0-EU-ES-03,"{""type"":""MultiPolygon"",""coordinates"":[[[[-1.09...",Alicante,38.365885,-0.427735,"1,543","{'es': '+1,78%', 'en': '+1.78%', 'it': '+1,78%...","{'es': '+28', 'en': '+28', 'it': '+28', 'pt': ...",#b1d004,31 de Julio,"[{'title': 'Días en mercado', 'value': 127}, {...",NaN,NaN


In [25]:
df_nested['index'].apply(lambda x: x.split('-')[2]).head()

0    ES
1    IT
2    ES
3    IT
4    ES
Name: index, dtype: object

In [16]:
df_nested['country'] = df_nested['index'].apply(lambda x: x.split('-')[2])

In [27]:
df_nested['wkt'].apply(lambda x: json.loads(x)).head()

0    {'type': 'MultiPolygon', 'coordinates': [[[[-9...
1    {'type': 'MultiPolygon', 'coordinates': [[[[12...
2    {'type': 'MultiPolygon', 'coordinates': [[[[-2...
3    {'type': 'MultiPolygon', 'coordinates': [[[[8....
4    {'type': 'MultiPolygon', 'coordinates': [[[[-1...
Name: wkt, dtype: object

In [28]:
# df_nested.drop(['wkt'], axis=1, inplace=True)
# df_nested['wkt'] = df_nested['wkt'].apply(lambda x: json.loads(x)) # it's a string
df_nested.drop(['priceThreeMTranslations', 'diferenceTranslations'], axis=1, inplace=True)
df_nested.dropna(subset=['chart'], inplace=True)
df_nested.head()

,index,wkt,province,latitude,longitude,price,color,closeDate,chart,province_chart,country_chart
0,0-EU-ES-15,"{""type"":""MultiPolygon"",""coordinates"":[[[[-9.30...",A Coruña,43.152465,-8.481585,"1,247",#b1d004,31 de Julio,"[{'title': 'Días en mercado', 'value': 177}, {...",NaN,NaN
1,0-EU-IT-AG,"{""type"":""MultiPolygon"",""coordinates"":[[[[12.31...",Agrigento,36.617789999999,13.179265,846,#cbe631,31 de Julio,"[{'title': 'Días en mercado', 'value': 45636},...",NaN,NaN
2,0-EU-ES-02,"{""type"":""MultiPolygon"",""coordinates"":[[[[-2.88...",Albacete,38.722695,-1.899265,"1,042",#cbe631,31 de Julio,"[{'title': 'Días en mercado', 'value': 188}, {...",NaN,NaN
3,0-EU-IT-AL,"{""type"":""MultiPolygon"",""coordinates"":[[[[8.109...",Alessandria,44.835174999999,8.6618049999998,839,#cbe631,31 de Julio,"[{'title': 'Días en mercado', 'value': 45632},...",NaN,NaN
4,0-EU-ES-03,"{""type"":""MultiPolygon"",""coordinates"":[[[[-1.09...",Alicante,38.365885,-0.427735,"1,543",#b1d004,31 de Julio,"[{'title': 'Días en mercado', 'value': 127}, {...",NaN,NaN


In [29]:
df_nested['Days on the market'] = df_nested.chart.apply(lambda x: x[0]['value'])
df_nested['Gross Yield'] = df_nested.chart.apply(lambda x: float(x[1]['value'].replace('%', '')))
df_nested['Turnover Rate'] = df_nested.chart.apply(lambda x: float(x[2]['value'].replace('%', '')))
df_nested.head()

,index,wkt,province,latitude,longitude,price,color,closeDate,chart,province_chart,country_chart,Days on the market,Gross Yield,Turnover Rate
0,0-EU-ES-15,"{""type"":""MultiPolygon"",""coordinates"":[[[[-9.30...",A Coruña,43.152465,-8.481585,"1,247",#b1d004,31 de Julio,"[{'title': 'Días en mercado', 'value': 177}, {...",NaN,NaN,177,4.46,3.34
1,0-EU-IT-AG,"{""type"":""MultiPolygon"",""coordinates"":[[[[12.31...",Agrigento,36.617789999999,13.179265,846,#cbe631,31 de Julio,"[{'title': 'Días en mercado', 'value': 45636},...",NaN,NaN,45636,5.74,3.08
2,0-EU-ES-02,"{""type"":""MultiPolygon"",""coordinates"":[[[[-2.88...",Albacete,38.722695,-1.899265,"1,042",#cbe631,31 de Julio,"[{'title': 'Días en mercado', 'value': 188}, {...",NaN,NaN,188,5.17,2.08
3,0-EU-IT-AL,"{""type"":""MultiPolygon"",""coordinates"":[[[[8.109...",Alessandria,44.835174999999,8.6618049999998,839,#cbe631,31 de Julio,"[{'title': 'Días en mercado', 'value': 45632},...",NaN,NaN,45632,4.94,4.57
4,0-EU-ES-03,"{""type"":""MultiPolygon"",""coordinates"":[[[[-1.09...",Alicante,38.365885,-0.427735,"1,543",#b1d004,31 de Julio,"[{'title': 'Días en mercado', 'value': 127}, {...",NaN,NaN,127,5.63,4.26


In [20]:
data_1 = df_nested.to_dict(orient='records', )
# data_1

In [30]:
final_df = json_normalize(data_1)
final_df.head()

,index,wkt,province,latitude,longitude,price,color,closeDate,chart,province_chart,country_chart,country,Days on the market,Gross Yield,Turnover Rate,province_chart.labels,province_chart.series,country_chart.labels,country_chart.series
0,0-EU-ES-15,"{""type"":""MultiPolygon"",""coordinates"":[[[[-9.30...",A Coruña,43.152465,-8.481585,"1,247",#b1d004,31 de Julio,"[{'title': 'Días en mercado', 'value': 177}, {...",NaN,NaN,ES,177,4.46,3.34,NaN,NaN,NaN,NaN
1,0-EU-IT-AG,"{""type"":""MultiPolygon"",""coordinates"":[[[[12.31...",Agrigento,36.617789999999,13.179265,846,#cbe631,31 de Julio,"[{'title': 'Días en mercado', 'value': 45636},...",NaN,NaN,IT,45636,5.74,3.08,NaN,NaN,NaN,NaN
2,0-EU-ES-02,"{""type"":""MultiPolygon"",""coordinates"":[[[[-2.88...",Albacete,38.722695,-1.899265,"1,042",#cbe631,31 de Julio,"[{'title': 'Días en mercado', 'value': 188}, {...",NaN,NaN,ES,188,5.17,2.08,NaN,NaN,NaN,NaN
3,0-EU-IT-AL,"{""type"":""MultiPolygon"",""coordinates"":[[[[8.109...",Alessandria,44.835174999999,8.6618049999998,839,#cbe631,31 de Julio,"[{'title': 'Días en mercado', 'value': 45632},...",NaN,NaN,IT,45632,4.94,4.57,NaN,NaN,NaN,NaN
4,0-EU-ES-03,"{""type"":""MultiPolygon"",""coordinates"":[[[[-1.09...",Alicante,38.365885,-0.427735,"1,543",#b1d004,31 de Julio,"[{'title': 'Días en mercado', 'value': 127}, {...",NaN,NaN,ES,127,5.63,4.26,NaN,NaN,NaN,NaN


In [31]:
columns_to_keep = ['wkt', 'country', 'province', 'Days on the market', 'Gross Yield', 'Turnover Rate']
final_df = final_df[columns_to_keep]
final_df.head()

,wkt,country,province,Days on the market,Gross Yield,Turnover Rate
0,"{""type"":""MultiPolygon"",""coordinates"":[[[[-9.30...",ES,A Coruña,177,4.46,3.34
1,"{""type"":""MultiPolygon"",""coordinates"":[[[[12.31...",IT,Agrigento,45636,5.74,3.08
2,"{""type"":""MultiPolygon"",""coordinates"":[[[[-2.88...",ES,Albacete,188,5.17,2.08
3,"{""type"":""MultiPolygon"",""coordinates"":[[[[8.109...",IT,Alessandria,45632,4.94,4.57
4,"{""type"":""MultiPolygon"",""coordinates"":[[[[-1.09...",ES,Alicante,127,5.63,4.26


In [23]:
final_df.to_csv('idealista_basic_data.csv')

# Add country to carto SQL

In [ ]:
# forgot to add it and added it later to try to improve geocoding for failed regions

In [35]:
sql_statement_template = """
UPDATE idealista_basic_data 
SET country = '${country}' 
WHERE province = '${province}';
"""

In [36]:
from string import Template

In [37]:
s = Template(sql_statement_template)

In [38]:
print(s.substitute(country='ES', province='Almeria'))


UPDATE idealista_basic_data 
SET country = 'ES' 
WHERE province = 'Almeria';



In [39]:
final_df.country.head()

0    ES
1    IT
2    ES
3    IT
4    ES
Name: country, dtype: object

In [40]:
with open('sql_country.txt', 'w') as f:
    f.writelines(final_df.apply(lambda x: s.substitute(country=x.country,
                                                       province=x.province.replace("'", "''")),
                                axis=1))

# Add geom to carto SQL

In [181]:
sql_statement_template = """
UPDATE idealista_basic_data 
SET the_geom = '${wkt}' 
WHERE province = '${province}';
"""

In [182]:
from string import Template

In [183]:
s = Template(sql_statement_template)

In [33]:
final_df.wkt.head()

0    {"type":"MultiPolygon","coordinates":[[[[-9.30...
1    {"type":"MultiPolygon","coordinates":[[[[12.31...
2    {"type":"MultiPolygon","coordinates":[[[[-2.88...
3    {"type":"MultiPolygon","coordinates":[[[[8.109...
4    {"type":"MultiPolygon","coordinates":[[[[-1.09...
Name: wkt, dtype: object

In [194]:
with open('sql_geo.txt', 'w') as f:
    f.writelines(final_df.apply(lambda x: s.substitute(
        wkt=x.wkt,
        province=x.province.replace("'", "''")),
                                axis=1))

In [201]:
final_df.head()

,wkt,country,province,Days on the market,Gross Yield,Turnover Rate
0,"{""type"":""MultiPolygon"",""coordinates"":[[[[-9.30...",ES,A Coruña,177,4.46,3.34
1,"{""type"":""MultiPolygon"",""coordinates"":[[[[12.31...",IT,Agrigento,45636,5.74,3.08
2,"{""type"":""MultiPolygon"",""coordinates"":[[[[-2.88...",ES,Albacete,188,5.17,2.08
3,"{""type"":""MultiPolygon"",""coordinates"":[[[[8.109...",IT,Alessandria,45632,4.94,4.57
4,"{""type"":""MultiPolygon"",""coordinates"":[[[[-1.09...",ES,Alicante,127,5.63,4.26
